In [1]:
from multiprocessing import Pool
import tqdm
import pyrosetta
import pandas as pd
import os
from pyrosetta.teaching import *
from scipy.stats import rankdata

pyrosetta.init("-mute all")

df = pd.read_csv("./data/main_dataset_creation/outputs/all_v3/submission_with_voxel_from_kaggle_filled_nan.csv")
scores = []
sfxn = get_score_function(True)



PyRosetta-4 2022 [Rosetta PyRosetta4.Release.python38.ubuntu 2022.41+release.28dc2a1757fe7c8bf9b661dd8d6cd903c305e581 2022-10-14T12:31:19] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.


In [2]:
df["mutated_rosetta_ref15_score"] = 0

def add_ref15(relaxed_mutated_3D_path):
    pose = pyrosetta.pose_from_pdb(relaxed_mutated_3D_path)
    return relaxed_mutated_3D_path, sfxn(pose)


paths = df["relaxed_mutated_3D_path"].tolist()

with Pool(12) as p:
    results = list(tqdm.tqdm(p.imap(add_ref15, paths), total=len(paths)))
# df["mutated_rosetta_ref15_score"] = df["relaxed_mutated_3D_path"].apply(
#     add_ref15)
# df.to_csv("test_ref15")

100%|██████████| 2413/2413 [02:17<00:00, 17.49it/s]


In [7]:
score_res_dict = dict(results)
df["mutated_rosetta_ref15_score"] = df["relaxed_mutated_3D_path"].apply(
    lambda x: -1*score_res_dict[x])
    


In [13]:
df["mutated_relaxed_total_score_rank"] = rankdata(df["mutated_relaxed_total_score"].apply(lambda x: -1*x))
df["mutated_rosetta_ref15_score_rank"] = rankdata(
    df["mutated_rosetta_ref15_score"])


In [14]:
df[["wild_aa", "mutation_position", "mutated_aa", "wild_relaxed_total_score",
    "mutated_relaxed_total_score", "mutated_rosetta_ref15_score", 
    "mutated_rosetta_ref15_score_rank", "mutated_relaxed_total_score_rank"]].head()


,wild_aa,mutation_position,mutated_aa,wild_relaxed_total_score,mutated_relaxed_total_score,mutated_rosetta_ref15_score,mutated_rosetta_ref15_score_rank,mutated_relaxed_total_score_rank
0,L,38,E,-430.177822,-472.490045,472.490057,2322.0,2322.0
1,L,38,K,-430.177822,-472.468226,472.468238,2321.0,2321.0
2,L,38,-,-430.177822,-511.424581,511.424592,2398.0,2398.0
3,K,39,C,-430.177822,-465.810497,465.810509,1148.0,1148.0
4,K,39,F,-430.177822,-467.134958,467.134970,1508.0,1508.0


In [ ]:
df["scores_rank"] = rankdata(df[["mutated_relaxed_total_score"]])
df[["mutated_relaxed_total_score", "scores_rank"]]



In [ ]:
for i in range(len(df)):
    if not i % 100:
        print(i)
    pose = pyrosetta.pose_from_pdb(os.path.join(
        relaxed_path, mutation_to_pdb_df.iloc[i]['path']))
    scores.append(sfxn(pose))

mutation_to_pdb_df['scores'] = scores
scores = mutation_to_pdb_df
scores.head()
